In [ ]:
import pandas as pd
import os

In [ ]:
pip install gdown

In [ ]:
!mkdir data

!wget -O data/indications_full_text.csv.gz https://huggingface.co/datasets/um-ids/dailymed-annotations/resolve/main/indications_full_text.csv.gz

# our use
#!gdown 15SXFzKx31xfiTKWs6vvFtq313q3UlTiw -O data/rxnorm_mappings.zip.zip

# For latest rxnorm mappings
!wget -O data/rxnorm_mappings.zip https://dailymed-data.nlm.nih.gov/public-release-files/rxnorm_mappings.zip

mkdir: cannot create directory ‘data’: File exists
--2023-11-03 11:16:02--  https://huggingface.co/datasets/um-ids/dailymed-annotations/resolve/main/indications_full_text.csv.gz
Resolving huggingface.co (huggingface.co)... 65.8.243.46, 65.8.243.92, 65.8.243.90, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.46|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c6/c6/c6c6f8f7ef3e0d8f747089dd0d053687bfa8747972aee98ba6fe2020d463f816/387d1da555c4f81e65232d2118f11dc494e85e724bc4f7c99ef75fbcafa9841c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27indications_full_text.csv.gz%3B+filename%3D%22indications_full_text.csv.gz%22%3B&response-content-type=application%2Fgzip&Expires=1699269362&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTI2OTM2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jNi9jNi9jNmM2ZjhmN2VmM2UwZDhmNzQ3MDg5ZGQwZDA1Mz

In [ ]:
# !unzip data/indications_full_text.zip -d data
!gzip -dk data/indications_full_text.csv.gz
!unzip data/rxnorm_mappings.zip -d data

gzip: data/indications_full_text.csv already exists; do you wish to overwrite (y or n)? y
Archive:  data/rxnorm_mappings.zip
replace data/rxnorm_mappings.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/rxnorm_mappings.txt  


In [ ]:
folder = 'data'
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 10)

rxm_header = dict({
    "SETID":"SetId",
    "SPL_VERSION":"",
    "RXCUI":"",
    "RXSTRING":"",
    "RXTTY":""
})

indications_full = dict({
    "set_id":"","xml_id":"","version_number":"","indication":""
})

In [ ]:
file = f'{folder}/indications_full_text.csv'
inds = pd.read_csv(f'{file}', sep=',', header=0, index_col=False, names=indications_full.keys())

file = f'{folder}/rxnorm_mappings.txt'
rxm = pd.read_csv(f'{file}', sep='|', header=0, index_col=False, names=rxm_header.keys())

There are 48,396 indication texts

In [ ]:
inds

,set_id,xml_id,version_number,indication
0,513ecb...,024c00...,1,\n\n\n...
1,4034fb...,feb790...,30,\n\n\n...
2,e06b3b...,c8d74a...,7,\n\n\n...
3,6ba2c7...,2f083a...,1,\n\n\n...
4,cc2353...,cc2353...,1,\n\n\n...
...,...,...,...,...
48391,0e7bca...,2e58de...,3,\n\n\n...
48392,fd3825...,593fed...,6,\n\n\n...
48393,98bae5...,d4bd0a...,7,\n\n\n...
48394,fc0cc9...,23972e...,3,\n\n\n...


In [ ]:
rxm

,SETID,SPL_VERSION,RXCUI,RXSTRING,RXTTY
0,000155...,27,198014,naprox...,PSN
1,000155...,27,198014,naprox...,SCD
2,0001d1...,2,1046593,benzal...,PSN
3,0001d1...,2,1046593,benzal...,SY
4,0001d1...,2,1046593,benzal...,SCD
...,...,...,...,...,...
457357,le7230...,1,209011,calciu...,SBD
457358,lf69f1...,1,836358,ipratr...,PSN
457359,lf69f1...,1,836358,ipratr...,SY
457360,lf69f1...,1,836358,ipratr...,SCD


Filter based on PSN

In [ ]:
rxm_psn = rxm.query('RXTTY=="PSN"')
len(rxm_psn)

146325

Take the first

In [ ]:
rxm_psn_first = rxm_psn.groupby('RXCUI').first()
len(rxm_psn_first)

23192

Remove the duplicated SET_ID's

In [ ]:
rxm_psn_first_non_dub = rxm_psn_first.drop_duplicates("SETID")
len(rxm_psn_first_non_dub)

11541

Add the indication text to filtered set_ids

In [ ]:
rxm_psn_first_non_dub = rxm_psn_first_non_dub.merge(inds, left_on='SETID', right_on='set_id', how='inner') # length 239195
rxm_psn_first_non_dub.reset_index(drop=True, inplace=True)
len(rxm_psn_first_non_dub)

5693

In [ ]:
# gp = p.query('RXTTY=="PSN"') # length: 82901
#gp = gp.groupby('RXCUI').first() # length: 14752
#gp = gp.drop_duplicates("SETID") # length: 5790

#len(rxm_psn_first)

Clean text and filter based on text length

In [ ]:
min_len = 75
max_len = 800
rxm_psn_first_non_dub['indication_length'] = rxm_psn_first_non_dub['indication'].str.len()
rxm_psn_first_non_dub_len = rxm_psn_first_non_dub[(rxm_psn_first_non_dub['indication_length'] >= min_len) & (rxm_psn_first_non_dub['indication_length'] <= 800)]
rxm_psn_first_non_dub_len.drop(columns=['indication_length'], inplace=True)

len(rxm_psn_first_non_dub_len)

<ipython-input-36-6bf81a223f5e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rxm_psn_first_non_dub_len.drop(columns=['indication_length'], inplace=True)


2913

In [ ]:
def clean(row):
    import re
    cleaned_text = row['indication']

    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)  # Remove extra line breaks
    cleaned_text = re.sub(r'\n\s+', '\n', cleaned_text)  # Remove leading spaces after line breaks
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'INDICATIONS AND USAGE:?','', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'INDICATIONS & USAGE','', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'INDICATION','', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'INIDCATIONS','', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'^\s?[\d:-]*(\.|\s)?','', cleaned_text)
    cleaned_text = re.sub(r'^[\d\.]*','', cleaned_text)
    cleaned_text = re.sub(r'^%','', cleaned_text)
    cleaned_text = cleaned_text.encode("ascii", "ignore").decode()

    return cleaned_text.strip()

rxm_psn_first_non_dub_len['clean'] = rxm_psn_first_non_dub_len.apply(lambda row : clean(row), axis=1)
g = rxm_psn_first_non_dub_len.drop(columns='indication')
g.to_csv(f"{folder}/group_by_PSN_RXCUI_nodup_spl_clean.csv")

<ipython-input-57-6865b41320c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rxm_psn_first_non_dub_len['clean'] = rxm_psn_first_non_dub_len.apply(lambda row : clean(row), axis=1)


In [ ]:
!wc -l data/group_by_PSN_RXCUI_nodup_spl_clean.csv

2914 data/group_by_PSN_RXCUI_nodup_spl_clean.csv
